In [1]:
import os, sys
notebook_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(notebook_dir,'..')))

In [3]:
%pip install setuptools

from scripts.data_load import load_data
from scripts.data_visualization import *
import pandas as pd
import numpy as np
import scorecardpy as sc

  Using cached setuptools-75.8.0-py3-none-any.whl.metadata (6.7 kB)
Using cached setuptools-75.8.0-py3-none-any.whl (1.2 MB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
df = load_data("../data/cleaned_dataset.csv")

In [5]:
df = df.rename(columns={"FraudResult": "target"})

In [6]:
# Replace missing values
df.fillna(-999, inplace=True)

In [12]:
# Convert non-numeric values to NaN and then fill with a placeholder
df = df.apply(pd.to_numeric, errors='coerce').fillna(-999)

# Automatic binning of numerical and categorical variables
bins = sc.woebin(df, y="target", bins=10, no_cores=2)

[INFO] creating woe binning ...


c:\Users\USER\Documents\10X Exercises\Week-6-Bati-Bank\.venv\Lib\site-packages\scorecardpy\condition_fun.py:34: UserWarning: There are 12 columns have only one unique values, which are removed from input dataset. 
 (ColumnNames: TransactionId, BatchId, AccountId, SubscriptionId, CustomerId, CurrencyCode, CountryCode, ProviderId, ProductId, ProductCategory, ChannelId, TransactionStartTime)
  warnings.warn("There are {} columns have only one unique values, which are removed from input dataset. \n (ColumnNames: {})".format(len(unique1_cols), ', '.join(unique1_cols)))
c:\Users\USER\Documents\10X Exercises\Week-6-Bati-Bank\.venv\Lib\site-packages\scorecardpy\condition_fun.py:40: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  datetime_cols = dat.apply(pd.to_numeric,errors='ignore').select_dtypes(object).apply(pd.to_datetime,errors='ignore').select_dtypes('datetime64').columns.t

In [13]:
print(bins['Amount'])

  variable           bin  count  count_distr   good  bad   badprob       woe  \
0   Amount    [-inf,0.0)  38189     0.399208  38184    5  0.000131 -2.736867   
1   Amount  [0.0,1500.0)  25243     0.263877  25242    1  0.000040 -3.932398   
2   Amount  [1500.0,inf)  32230     0.336915  32043  187  0.005802  1.060141   

     bin_iv  total_iv  breaks  is_special_values  
0  1.023740  2.714451     0.0              False  
1  1.019351  2.714451  1500.0              False  
2  0.671360  2.714451     inf              False  


In [14]:
# Apply WOE transformations
df_woe = sc.woebin_ply(df, bins)

[INFO] converting into woe values ...


In [15]:
# Display the transformed dataset with WOE values
print(df_woe.head())

   TransactionId  CountryCode  TransactionStartTime  CurrencyCode  target  \
0         -999.0          256                -999.0        -999.0       0   
1         -999.0          256                -999.0        -999.0       0   
2         -999.0          256                -999.0        -999.0       0   
3         -999.0          256                -999.0        -999.0       0   
4         -999.0          256                -999.0        -999.0       0   

   AccountId  CustomerId  ProviderId  ProductId  ChannelId  SubscriptionId  \
0     -999.0      -999.0      -999.0     -999.0     -999.0          -999.0   
1     -999.0      -999.0      -999.0     -999.0     -999.0          -999.0   
2     -999.0      -999.0      -999.0     -999.0     -999.0          -999.0   
3     -999.0      -999.0      -999.0     -999.0     -999.0          -999.0   
4     -999.0      -999.0      -999.0     -999.0     -999.0          -999.0   

   BatchId  ProductCategory  Amount_woe  Value_woe  PricingStrategy_

In [17]:
# Extract the IV values for all features
iv_values = {key: value['total_iv'].values[0] for key, value in bins.items()}

In [18]:
# Display IV summary
print("Information Value (IV):")
print(iv_values)

Information Value (IV):
{'Amount': np.float64(2.714451218724242), 'Value': np.float64(2.6411904776462554), 'PricingStrategy': np.float64(0.08552868425952848)}


In [20]:
# ------------------------
# Save WOE-Transformed Dataset
# ------------------------
df_woe.to_csv('../data/woe_transformed_data.csv', index=False)
print("WOE-transformed dataset saved as 'woe_transformed_data.csv'.")

# ------------------------
# Save Information Value (IV)
# ------------------------
iv_values_df = pd.DataFrame(list(iv_values.items()), columns=['Variable', 'IV'])
iv_values_df.to_csv('../data/information_values.csv', index=False)
print("Information Value (IV) summary saved as 'information_values.csv'.")

WOE-transformed dataset saved as 'woe_transformed_data.csv'.
Information Value (IV) summary saved as 'information_values.csv'.
